In [1]:
import pandas as pd

In [2]:
all_merged = pd.DataFrame({'category_type': [], 'category': [], 'inflation_without_shelter': [], 'inflation_with_shelter': []})


# Deciles

In [3]:
longNameOfDiscriminatingColumn = "decile"
discriminatingColumn = "DECUC"

In [4]:
df_106 = pd.read_csv("computed_data/106_inflation_March_2022.csv")
df_106_avoided = pd.read_csv("computed_data/106_inflation_avoided.csv")

In [5]:
merged = df_106.merge(right=df_106_avoided, on=discriminatingColumn)

In [6]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [7]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [8]:
all_merged = pd.concat([merged, all_merged])

# Age

In [9]:
translations = {'1': 'Less than 25 years',
                '2': '25-34 years',
                '3': '35-44 years',
                '4': '45-54 years',
                '5': '55-64 years',
                '6': '65-74 years',
                '7': '75 years or older'}
TFnumber = 102
discriminatingColumn = "AGPR"
longNameOfDiscriminatingColumn = "age"

In [10]:

df = pd.read_csv(f"computed_data/{TFnumber}_inflation_March_2022.csv")
df_avoided = pd.read_csv(f"computed_data/{TFnumber}_inflation_avoided.csv")

In [11]:
df_avoided[discriminatingColumn] = df_avoided[discriminatingColumn].astype(str).replace(translations)

In [12]:
merged = df.merge(right=df_avoided, on=discriminatingColumn)

In [13]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [14]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [15]:
all_merged = pd.concat([merged, all_merged])

# Socioeconomic category

In [ ]:
TFnumber = '103'
# create a dictionary with the translations
longNameOfDiscriminatingColumn = "socioeconomic category"
# create a dictionary with the translations
translations = {
    "1": "Farmers",
    "2": "Craftsmen, traders and business owners",
    "3": "Executives",
    "4": "Intermediate professions",
    "5": "Employees",
    "6": "Workers",
    "7": "Retirees",
    "8": "Other inactive"
}
discriminatingColumn = "CSPR"

In [ ]:

df = pd.read_csv(f"computed_data/{TFnumber}_inflation_March_2022.csv")
df_avoided = pd.read_csv(f"computed_data/{TFnumber}_inflation_avoided.csv")

In [ ]:
df_avoided[discriminatingColumn] = df_avoided[discriminatingColumn].astype(str).replace(translations)

In [ ]:
merged = df.merge(right=df_avoided, on=discriminatingColumn)

In [ ]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [ ]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [ ]:
all_merged = pd.concat([merged, all_merged])

# Category of residential municipality

In [ ]:
longNameOfDiscriminatingColumn = "category of residential municipality"
TFnumber = "104"
discriminatingColumn = "STRATE"
translations = {'0': 'Rural', '1': 'Small-sized towns', '2': 'Medium-sized towns', '3': 'Large cities', '4': 'Paris metropolitan area'}


In [ ]:

df = pd.read_csv(f"computed_data/{TFnumber}_inflation_March_2022.csv")
df_avoided = pd.read_csv(f"computed_data/{TFnumber}_inflation_avoided.csv")

In [ ]:
df_avoided[discriminatingColumn] = df_avoided[discriminatingColumn].astype(str).replace(translations)

In [ ]:
merged = df.merge(right=df_avoided, on=discriminatingColumn)

In [ ]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [ ]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [ ]:
all_merged = pd.concat([merged, all_merged])

# Household type

In [ ]:
TFnumber = '105'
# create a dictionary with the translations
longNameOfDiscriminatingColumn = "household type"
# create a dictionary with the translations
translations = {'1': 'Single-person households',
                '2': 'Single-parent families',
                '3': 'Couples without children',
                '4': 'Couples with children',
                '5': 'Other households'}

discriminatingColumn = "TYPMEN"

In [ ]:

df = pd.read_csv(f"computed_data/{TFnumber}_inflation_March_2022.csv")
df_avoided = pd.read_csv(f"computed_data/{TFnumber}_inflation_avoided.csv")

In [ ]:
df_avoided[discriminatingColumn] = df_avoided[discriminatingColumn].astype(str).replace(translations)

In [ ]:
merged = df.merge(right=df_avoided, on=discriminatingColumn)

In [ ]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [ ]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [ ]:
all_merged = pd.concat([merged, all_merged])

# Occupancy status of the main residence

In [ ]:
TFnumber = '107'
# create a dictionary with the translations
longNameOfDiscriminatingColumn = "occupancy status of the main residence"
# create a dictionary with the translations
translations = {'P': 'Owner',
                'L': 'Renter'}


discriminatingColumn = "STATUT"

In [ ]:

df = pd.read_csv(f"computed_data/{TFnumber}_inflation_March_2022.csv")
df_avoided = pd.read_csv(f"computed_data/{TFnumber}_inflation_avoided.csv")

In [ ]:
df_avoided[discriminatingColumn] = df_avoided[discriminatingColumn].astype(str).replace(translations)

In [ ]:
merged = df.merge(right=df_avoided, on=discriminatingColumn)

In [ ]:
merged["inflation_without_shelter"] = merged['weight_Group_inflation_2022_03'] + merged['inflation_avoided']

In [ ]:
merged = merged.rename(columns={discriminatingColumn: 'category_type', 'weight_Group_inflation_2022_03': 'inflation_with_shelter'})
merged['category'] = longNameOfDiscriminatingColumn

In [ ]:
all_merged = pd.concat([merged, all_merged])

# Chart